# Image Classification Live Inference

Check connected video device address

In [ ]:
!ls -ltrh /dev/video*

Run the camera

In [ ]:
from jetcam.csi_camera import CSICamera
from jetcam.usb_camera import USBCamera

# USB Camera
camera1 = USBCamera(width=224, height=224, capture_device=1) # confirm the capture_device number
#camera = USBCamera(width=244, height=244, capture_width=640, capture_height=480, capture_de)

# CSI Camera (Raspberry Pi Camera Module V2)
camera = CSICamera(width=224, height=224, capture_device=0)
#camera1 = CSICamera(width=244, height=244, capture_device=1)

camera.running = True
camera1.running = True
print("camera created")

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')
image_widget2 = ipywidgets.Image(format='jpeg')
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
camera_link2 = traitlets.dlink((camera1, 'value'), (image_widget2, 'value'), transform=bgr8_to_jpeg)
#display(image_widget)
#display(image_widget2)

Uncomment/edit the associated lines for the classification task you're building and execute the cell.
This cell should only take a few seconds to execute.

### Model

In [ ]:
import torch
device = torch.device('cuda')

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('/nvdli-nano/data/classification/my_model_trt.pth'))

### Live  Execution
Execute the cell below to set up the live execution widget.  This cell should only take a few seconds to execute.

In [ ]:
import threading
import time
from utils import preprocess
import torch.nn.functional as F

CATEGORIES = ['background', 'bluecar', 'yellowcar']

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
state_widget1 = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='prediction')
prediction_widget1 = ipywidgets.Text(description='prediction')
score_widgets = []
score_widgets1 = []
for category in CATEGORIES:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widget1 = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)
    score_widgets1.append(score_widget1)

def live(state_widget, model_trt, camera, prediction_widget, score_widget):
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model_trt(preprocessed)
        output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = output.argmax()
        prediction_widget.value = CATEGORIES[category_index]
        for i, score in enumerate(list(output)):
            score_widgets[i].value = score  

def live1(state_widget1, model_trt, camera1, prediction_widget1, score_widget1):
    while state_widget1.value == 'live':
        image1 = camera1.value
        preprocessed1 = preprocess(image1)
        output1 = model_trt(preprocessed1)
        output1 = F.softmax(output1, dim=1).detach().cpu().numpy().flatten()
        category_index1 = output1.argmax()
        prediction_widget1.value = CATEGORIES[category_index1]
        for i, score1 in enumerate(list(output1)):
            score_widgets1[i].value = score1 

def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model_trt, camera, prediction_widget, score_widget))
        execute_thread.start()
        
def start_live1(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live1, args=(state_widget1, model_trt, camera1, prediction_widget1, score_widget1))
        execute_thread.start() 
        
state_widget.observe(start_live, names='value')
state_widget1.observe(start_live1, names='value')

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets),
    prediction_widget,
    state_widget
])

live_execution_widget1 = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets1),
    prediction_widget1,
    state_widget1
])

print("live_execution_widget created")

### Display the Interactive Tool!

In [ ]:
first_camera = ipywidgets.HBox([image_widget, live_execution_widget])
second_camera = ipywidgets.HBox([image_widget2, live_execution_widget1])

print("Drive-thru camera 1")
display(first_camera)
print("Drive-thru camera 2")
display(second_camera)

In [ ]:
camera.running = False
#camera.cap.release()

In [ ]:
import os
os._exit(00)